<a href="https://colab.research.google.com/github/acidrefluxreflex/acidrefluxreflex/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIと会話する

## 概要

雑談できる人工知能は人類の夢。ということで自然言語処理を使って会話できるAIを作成した。会話の精度は改善の余地があるが、どんな話題でも一応対話することはできる。

  

## 技術

参考記事は関数として機能を実装していたのでクラスに書き換えておいた。また字数制限によるエラーもあったので修正し、文章を生成する`generate()`関数を切り出して汎用性も高めておいた。





---

## コード解説
まずは必要ライブラリを準備する。


In [1]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.4 MB/s 
     |████████████████████████████████| 596 kB 59.0 MB/s 
     |████████████████████████████████| 101 kB 12.4 MB/s 
     |████████████████████████████████| 6.6 MB 49.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 4.9 MB/s 


In [ ]:
from transformers import AutoModelForCausalLM
model_name = "rinna/japanese-gpt-1b"
model = AutoModelForCausalLM.from_pretrained(model_name)

In [3]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/283 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/153 [00:00<?, ?B/s]

KeyboardInterrupt: ignored

In [6]:
import torch

## チャットボット
会話をする部分はクラスとして実装した。文章生成と会話をそれぞれ行う２つの関数を持っている。

In [16]:
class ChatBot(torch.nn.Module):

   def __init__(self):
         super(ChatBot, self).__init__()
         

   #文章生成を行う関数。元になる文章、最大文字数、最小文字数を引数にもつ。
   def generate(self, text, max_length, min_length):
     token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")
     with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            max_length=max_length,
            min_length=min_length,
            do_sample=True,
            top_k=500,
            top_p=0.95,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            bad_word_ids=[[tokenizer.unk_token_id]]
        )
        output = tokenizer.decode(output_ids.tolist()[0])
        return output
    

   def chat(self):
     #プロフィール設定
     name = input("AIの名前:")
     name_text = f"あなたは{name}です。あなたの名前は{name}といいます。"
     hobby = input("AIの趣味:")
     hobby_text = f"あなたの趣味は{hobby}で、休日は{hobby}をして過ごしています。"
     work = input("AIの職業:")
     work_text = f"あなたの職業は{work}で、普段は{work}として生活しています。"

     print("AIに言いたい事を入力してください。終了したいときは未入力のままEnter")
     userInput = "ッ"
     text = name_text + hobby_text + work_text + f"以下は人間とあなたの会話です。人間:「こんにちは!」あなた:「はい、こんにちは」人間:「"
     max_length = 70
     min_length = 40
     

     while userInput != "":
       userInput = input(">>> ")
       if userInput == "":
           print("会話を終了します")
           break
       text += userInput + f"」あなた:「"

       #文字数調節
       message_gap = len(text) - max_length
       if message_gap > 40:
         max_length = len(text) - 30

       output = self.generate(text,max_length,min_length)

       #半角正則化
       text = text.translate(str.maketrans({chr(0xFF01 + i): chr(0x21 + i) for i in range(94)}))
       #今回の応答より前をもぎ取る
       output = output.replace(text, "")
       #最初の」までを切り取る
       outputList = []
       for l in output:
        outputList.append(l)
        if l == "」":
            break
       outputSentence = "".join(outputList)
       text += outputSentence + "人間:「"
       message = outputSentence.replace("」", "")

       #文字数調節
       max_length = len(message) + 70
       min_length = len(message) + 40
      
       print(message)
      

## 実際に会話してみる

最初にAIの名前、趣味、基本的にはLINEやメッセージアプリのように返信をしていく。会話を終わらせたい時は入力なしでEnterを押せばよい。まるで既読無視だね。


In [17]:
bot = ChatBot()

In [ ]:
bot.chat()

## 改善点
やはりAI側の初期設定が問題だろう。より軽量のモデルとファインチューニングを組み合わせれば名前や口調を自由に変えられる。会話が長引くと、読み取る文脈も長くなって処理に時間がかかる問題もある。


## 参考



[13億パラメータのGPT日本語学習済みモデルが出たので会話応答を生成してみた - Qiita](https://qiita.com/MamoruItoi/items/a18abfedb79a57aeb91c)

